In [1]:
from dariah_topics import preprocessing as pre
from dariah_topics import visualization as visual
from dariah_topics import mallet as mal
from dariah_topics import evaluation

/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationW

## Preprocessing

#### Liste mit Dateinamen erzeugen

In [2]:
path_txt = "corpus_txt"
#path_txt = "grenzbote_plain/*/"

doclist_txt = pre.create_document_list(path_txt)
assert doclist_txt, "No documents found"
doclist_txt[:5]

['corpus_txt/Poe_ThePurloinedLetter.txt',
 'corpus_txt/Lovecraft_TheShunnedHouse.txt',
 'corpus_txt/Kipling_ThyServantaDog.txt',
 'corpus_txt/Howard_ShadowsintheMoonlight.txt',
 'corpus_txt/Howard_SchadowsinZamboula.txt']

In [3]:
path_csv = "corpus_csv"

doclist_csv = pre.create_document_list(path_csv, 'csv')
doclist_csv[:5]

['corpus_csv/Howard_ShadowsintheMoonlight.txt.csv',
 'corpus_csv/Lovecraft_AttheMountainofMadness.txt.csv',
 'corpus_csv/Poe_TheMasqueoftheRedDeath.txt.csv',
 'corpus_csv/Doyle_AStudyinScarlet.txt.csv',
 'corpus_csv/Poe_TheCaskofAmontillado.txt.csv']

In [4]:
path_tei = "corpus_tei"

doclist_tei = pre.create_document_list(path_tei, 'xml')
doclist_tei[:5]

['corpus_tei/Schnitzler_DerAndere.xml',
 'corpus_tei/Schnitzler_GeschichteeinesGenies.xml',
 'corpus_tei/Schnitzler_DieFremde.xml',
 'corpus_tei/Schnitzler_Amerika.xml',
 'corpus_tei/Schnitzler_DerWitwer.xml']

#####  Liste mit Dokumentenlabels erzeugen - (Funktion wird durch Thorsten's generischere Funktion ersetzt)

In [5]:
doc_labels = list(pre.get_labels(doclist_txt))
doc_labels[:5]

['corpus_txt/Poe_ThePurloinedLetter.txt',
 'corpus_txt/Lovecraft_TheShunnedHouse.txt',
 'corpus_txt/Kipling_ThyServantaDog.txt',
 'corpus_txt/Howard_ShadowsintheMoonlight.txt',
 'corpus_txt/Howard_SchadowsinZamboula.txt']

#### Corpus laden

In [6]:
corpus_txt = pre.read_from_txt(doclist_txt)

In [7]:
corpus_csv = pre.read_from_csv(doclist_csv)

In [8]:
corpus_tei = pre.read_from_tei(doclist_tei)

#### Tokenisieren

In [9]:
doc_tokens = pre.filter_POS_tags(corpus_csv)
list(doc_tokens)[0][:5]

24    extensive
44        swift
50         tall
54        heavy
81         dark
Name: Lemma, dtype: object

In [10]:
doc_tokens = [list(pre.tokenize(txt)) for txt in list(corpus_txt)]
doc_tokens[0][:5]

['the', 'purloined', 'letter', 'edgar', 'allan']

In [11]:
doc_tokens = [list(pre.tokenize(tei)) for tei in list(corpus_tei)]
doc_tokens[0][:5]

['arthur', 'schnitzler', 'der', 'andere', 'aus']

#### Create Dictionaries

In [12]:
id_types, doc_ids = pre.create_dictionaries(doc_labels, doc_tokens)

#### Sparse BOW Model

In [13]:
sparse_bow = pre.create_mm(doc_labels, doc_tokens, id_types, doc_ids)

In [14]:
sparse_bow[:5]

0
doc_id token_id   
1      4         1
       2053      1
       2055      1
       2056      2
       2059      1

#### Convert to Doc-Topic Matrix

In [15]:
'''import pandas as pd

doc2id = {value : key for key, value in doc_ids.items()}
type2id = {value : key for key, value in id_types.items()}

cols = [doc2id[key] for key in set(sparse_bow.index.get_level_values("doc_id"))]
#idx = [type2id[key] for key in set(sparse_bow.index.get_level_values("token_id"))]

set(sparse_bow.index.get_level_values("token_id"))

#doctopic_matrix = pd.DataFrame(columns=cols, index=idx)'''

'import pandas as pd\n\ndoc2id = {value : key for key, value in doc_ids.items()}\ntype2id = {value : key for key, value in id_types.items()}\n\ncols = [doc2id[key] for key in set(sparse_bow.index.get_level_values("doc_id"))]\n#idx = [type2id[key] for key in set(sparse_bow.index.get_level_values("token_id"))]\n\nset(sparse_bow.index.get_level_values("token_id"))\n\n#doctopic_matrix = pd.DataFrame(columns=cols, index=idx)'

#### Remove Features

In [16]:
import os.path
basepath = os.path.abspath('.')

with open(os.path.join(basepath, "tutorial_supplementals", "stopwords", "en.txt"), 'r', encoding = 'utf-8') as f: 
    stopword_list = f.read().split('\n')
    
stopword_list = set(stopword_list)

In [17]:
sparse_df_stopwords_removed = pre.remove_features(sparse_bow, id_types, stopword_list)

In [18]:
print(len(sparse_bow))
print(len(sparse_df_stopwords_removed))

2989
2948


##### Create Feature Remove Lists

In [19]:
stopword_list = pre.find_stopwords(sparse_bow, id_types, 100)
len(stopword_list)

100

In [20]:
hapax_list = pre.find_hapax(sparse_bow, id_types)
len(hapax_list)

1877

##### Remove Features

In [21]:
feature_list = set(stopword_list).union(hapax_list)
clean_term_frequency = pre.remove_features(sparse_bow, id_types, feature_list)

In [22]:
len(clean_term_frequency)

348

## Topic Modeling with Gensim

##### Saving Sparse BOW

In [23]:
num_docs = max(sparse_bow.index.get_level_values("doc_id"))
num_types = max(sparse_bow.index.get_level_values("token_id"))
sum_counts = sum(sparse_bow[0])

header_string = str(num_docs) + " " + str(num_types) + " " + str(sum_counts) + "\n"

with open("gb_plain.mm", 'w', encoding = "utf-8") as f:
    pass

with open("gb_plain.mm", 'a', encoding = "utf-8") as f:
    f.write("%%MatrixMarket matrix coordinate real general\n")
    f.write(header_string)
    sparse_bow.to_csv( f, sep = ' ', header = None)

#### Import Market Matrix

In [24]:
from gensim.models import LdaModel
from gensim.corpora import MmCorpus

In [25]:
mm = MmCorpus("gb_plain.mm")

In [26]:
#in case you're only loading the corpus - build dict first

#import pickle

#pickle.dump( id_types, open( "gb_plain.dictionary", "wb" ) )

In [27]:
#id_types = pickle.load(open("gb_plain.dictionary", 'rb'))

In [28]:
#mm = gensim.corpora.MmCorpus("gb_all.mm")

In [29]:
#id_types = pickle.load(open("gb_all.dictionary", 'rb'))

In [30]:
doc2id = {value : key for key, value in doc_ids.items()}
type2id = {value : key for key, value in id_types.items()}

In [31]:
#model = LdaModel(corpus=mm, id2word=type2id, num_topics=60, alpha = "symmetric", passes = 10) #import momentan in visual 
# -> da ich mir noch nicht sicher bin, welche Funktionen in das tm_gensim.py sollen
model = LdaModel(corpus=mm, id2word=type2id, num_topics=10)

WARNING gensim.models.ldamodel: too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy


In [32]:
#Anzeige der keywords für topic n
n = 1
topic_nr_x = model.get_topic_terms(n)

topicTerms = [type2id[i[0]] for i in topic_nr_x]
topicTerms

['durchdrungen',
 'weitergehen',
 'andre',
 'verlieh',
 'so',
 'geradeso',
 'treffen',
 'durch',
 'ernsteren',
 'vornehmheit']

In [33]:
#Anzeige der id des ersten keywords aus dem topic
'''n = 0
for key, value in type2id.items():
    if value == topicTerms[n]:
        print(key)'''

'n = 0\nfor key, value in type2id.items():\n    if value == topicTerms[n]:\n        print(key)'

In [34]:
#Gensim-Funktion -> Anzeige des word_id-probability-mappings eines topics n
'''n = 0
topicN = model.get_topic_terms(n)
topicN'''

'n = 0\ntopicN = model.get_topic_terms(n)\ntopicN'

In [35]:
#Topic probabilities
topics = model.show_topics(num_topics = 10)
topics[:2]

[(0,
  '0.030*"durchdrungen" + 0.022*"weitergehen" + 0.021*"verlieh" + 0.020*"so" + 0.017*"andre" + 0.015*"treffen" + 0.015*"geradeso" + 0.011*"vornehmheit" + 0.011*"herren" + 0.009*"ernsteren"'),
 (1,
  '0.020*"durchdrungen" + 0.018*"weitergehen" + 0.016*"andre" + 0.014*"verlieh" + 0.013*"so" + 0.012*"geradeso" + 0.009*"treffen" + 0.008*"durch" + 0.008*"ernsteren" + 0.008*"vornehmheit"')]

In [36]:
from scipy.io import mmread
test = mmread("gb_plain.mm")
import pandas as pd
pd.DataFrame(test.data)

,0
0,1
1,1
2,1
3,2
4,1
5,1
6,1
7,2
8,1
9,3


## Topic Modeling mit Mallet

##### Mallet Binary erzeugen

In [37]:
'''import os

#Variablen zum Testen
path_to_mallet = "/home/sina/Uni/Dariah/mallet/bin/mallet"

path_to_corpus = os.path.join(os.path.abspath('.'), 'corpus_txt')
assert os.path.exists(path_to_corpus)

malletBinary = mal.create_mallet_binary(path_to_corpus, path_to_mallet)
malletBinary'''

'import os\n\n#Variablen zum Testen\npath_to_mallet = "/home/sina/Uni/Dariah/mallet/bin/mallet"\n\npath_to_corpus = os.path.join(os.path.abspath(\'.\'), \'corpus_txt\')\nassert os.path.exists(path_to_corpus)\n\nmalletBinary = mal.create_mallet_binary(path_to_corpus, path_to_mallet)\nmalletBinary'

##### Mallet output erzeugen

In [38]:
'''outfolder = os.path.join(os.path.abspath('.'), "tutorial_supplementals/mallet_output")
mal.create_mallet_model(malletBinary,outfolder)'''

'outfolder = os.path.join(os.path.abspath(\'.\'), "tutorial_supplementals/mallet_output")\nmal.create_mallet_model(malletBinary,outfolder)'

##### Matrix von Mallet output erzeugen

In [39]:
'''basepath = os.path.join(os.path.abspath('.'), "tutorial_supplementals/mallet_output")
doc_topics = os.path.join(basepath, "doc_topics.txt")'''

'basepath = os.path.join(os.path.abspath(\'.\'), "tutorial_supplementals/mallet_output")\ndoc_topics = os.path.join(basepath, "doc_topics.txt")'

In [40]:
'''doctopicsMatrix = mal.create_MalletMatrix(doc_topics)'''

'doctopicsMatrix = mal.create_MalletMatrix(doc_topics)'

In [41]:
'''import pandas as pd
pd.DataFrame(doctopicsMatrix)'''

'import pandas as pd\npd.DataFrame(doctopicsMatrix)'

## Visualisierung 

In [42]:
'''lda_model = 'out_easy/corpus.lda'
corpus = 'out_easy/corpus.mm'
dictionary = 'out_easy/corpus.dict'
doc_labels = 'out_easy/corpus_doclabels.txt'
interactive  = False

vis = visual.Visualization(lda_model, corpus, dictionary, doc_labels, interactive)'''

"lda_model = 'out_easy/corpus.lda'\ncorpus = 'out_easy/corpus.mm'\ndictionary = 'out_easy/corpus.dict'\ndoc_labels = 'out_easy/corpus_doclabels.txt'\ninteractive  = False\n\nvis = visual.Visualization(lda_model, corpus, dictionary, doc_labels, interactive)"

#### Heatmap

In [43]:
#heatmap = visual.make_heatmap()

In [44]:
#visual.save_heatmap("./visualizations/heatmap")

#### Interactive

In [45]:
#vis = collection.Visualization(lda_model, corpus, dictionary, doc_labels, interactive=True)

In [46]:
#vis.make_interactive()

In [47]:
#vis.save_interactive("./visualizations/interactive")

## Evaluation

In [48]:
segmented_topics = evaluation.topic_segmenter(model, type2id, 10)
segmented_topics[0][:5]

[('durchdrungen', 'weitergehen'),
 ('durchdrungen', 'verlieh'),
 ('durchdrungen', 'so'),
 ('durchdrungen', 'andre'),
 ('durchdrungen', 'treffen')]

In [49]:
umass_probability = evaluation.token_probability(doc_tokens, segmented_topics)
umass_probability

geradeso                    {2}
weitergehen                 {0}
vornehmheit                 {2}
durch           {0, 1, 2, 3, 4}
gleicher                    {0}
schuld                      {4}
andre                    {1, 2}
verlieh                     {2}
rose                        {1}
gesucht                     {1}
herren                      {2}
ernsteren                   {4}
durchdrungen                {2}
ungewisse                   {2}
treffen                     {3}
so              {0, 1, 2, 3, 4}
dtype: object

In [50]:
umass = evaluation.calculate_umass(segmented_topics, umass_probability, doc_tokens, 10)
umass

-0.026952470310384204

In [51]:
wiki_list = evaluation.wikipedia_table_crawler()

/usr/lib/python3/dist-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)


In [52]:
wiki_corpus = evaluation.wikipedia_crawler(wiki_list, size=5)

In [53]:
uci_probability = evaluation.token_probability(wiki_corpus, segmented_topics)

In [54]:
pmi = evaluation.calculate_pointwise_mutual_information(segmented_topics, wiki_corpus, uci_probability)

In [55]:
uci = evaluation.calculate_uci(pmi, wiki_corpus, 10)
uci

0.0

![success](http://cdn2.hubspot.net/hub/128506/file-446943132-jpg/images/computer_woman_success.jpg)